<a href="https://colab.research.google.com/github/iamkanhaiyakumar/PPE_detection_Kit/blob/main/ppe_dectections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Nov 24 08:32:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.3/887.3 kB 27.9 MB/s eta 0:00:00


In [ ]:
!pip install opencv-python

In [ ]:
!pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26298 sha256=4b1214603b7588e2de60fe5611d2587a38872ca33b4070f5275894470e5a3fc1
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone


In [ ]:
from ultralytics import YOLO
import cv2
import cvzone
import math

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!yolo task=detect mode=predict model=yolov8l.pt conf=0.25 source='https://ultralytics.com/images/bus.jpg'

100% 83.7M/83.7M [00:00<00:00, 316MB/s]
Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8l summary (fused): 268 layers, 43,668,288 parameters, 0 gradients, 165.2 GFLOPs

100% 134k/134k [00:00<00:00, 6.21MB/s]
image 1/1 /content/bus.jpg: 640x480 5 persons, 1 bicycle, 1 bus, 137.3ms
Speed: 15.2ms preprocess, 137.3ms inference, 684.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!yolo task=detect mode=train model=yolov8l.pt data=/content/drive/MyDrive/Datasets/ConstructionSafety/data.yaml epochs=30 imgsz=640

Ultralytics 8.3.36 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/content/drive/MyDrive/Datasets/ConstructionSafety/data.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_label

In [ ]:
data = "/content/drive/MyDrive/Datasets/ConstructionSafety/data.yaml"

In [ ]:
import os
print(os.path.exists(data))


True


In [ ]:
import os

# Define paths
paths = {
    "Train": "/content/drive/MyDrive/Datasets/ConstructionSafety/train",
    "Test": "/content/drive/MyDrive/Datasets/ConstructionSafety/test",
    "Valid": "/content/drive/MyDrive/Datasets/ConstructionSafety/valid"
}

# Count files in each directory
for key, path in paths.items():
    num_files = sum([len(files) for _, _, files in os.walk(path)])
    print(f"{key}: {num_files} files")


Train: 5221 files
Test: 164 files
Valid: 229 files


In [ ]:
import os

# Define dataset paths
base_path = "/content/drive/MyDrive/Datasets/ConstructionSafety"
train_img_path = os.path.join(base_path, "train/images")
test_img_path = os.path.join(base_path, "test/images")
valid_img_path = os.path.join(base_path, "valid/images")

# Count images in each folder
train_count = len(os.listdir(train_img_path))
test_count = len(os.listdir(test_img_path))
valid_count = len(os.listdir(valid_img_path))

print(f"Training Images: {train_count}")
print(f"Testing Images: {test_count}")
print(f"Validation Images: {valid_count}")

# Count helmet and head images (Assuming labels are in .txt format)
helmet_count = 0
head_count = 0

train_label_path = os.path.join(base_path, "train/labels")

for label_file in os.listdir(train_label_path):
    with open(os.path.join(train_label_path, label_file), 'r') as f:
        for line in f:
            class_id = int(line.split()[0])  # Assuming first value is class ID
            if class_id == 0:  # Change class ID according to your dataset
                helmet_count += 1
            elif class_id == 1:
                head_count += 1

print(f"Helmet Images: {helmet_count}")
print(f"Head Images: {head_count}")


Training Images: 2615
Testing Images: 82
Validation Images: 114
Helmet Images: 3145
Head Images: 1651


In [ ]:
# Display the contents of data.yaml
with open('/content/drive/MyDrive/Datasets/ConstructionSafety/data.yaml', 'r') as file:
    print(file.read())


path: ../drive/MyDrive/Datasets/ConstructionSafety
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 10
names: ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']

roboflow:
  workspace: roboflow-universe-projects
  project: construction-site-safety
  version: 28
  license: CC BY 4.0
  url: https://universe.roboflow.com/roboflow-universe-projects/construction-site-safety/dataset/28


In [ ]:
!pip install roboflow ultralytics
!pip install --upgrade pip
!pip cache purge
!pip install roboflow ultralytics --no-cache-dir

from roboflow import Roboflow

rf = Roboflow(api_key="KPZSDSQA5hYxInevF3pj")
project = rf.workspace("roboflow-universe-projects").project("construction-site-safety")
dataset = project.version(28).download("yolov8")


Files removed: 2 (37 kB)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Construction-Site-Safety-28 in yolov8:: 100%|██████████| 5614/5614 [00:00<00:00, 5656.80it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
from ultralytics import YOLO

# Load the YOLO model
model = YOLO("yolov8n.pt")


# Train the model
model.train(data="/content/Construction-Site-Safety-28/data.yaml", epochs=10, imgsz=640)


Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Construction-Site-Safety-28/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_

train: Scanning /content/Construction-Site-Safety-28/train/labels.cache... 2605 images, 6 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/004720_jpg.rf.afc486560a4004c7cfd67910af31a29c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/construction-813-_jpg.rf.b085952261fd98f2e76b8065de149b5f.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Construction-Site-Safety-28/valid/labels.cache... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        114        697        0.8      0.607       0.68       0.36



10 epochs completed in 0.137 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]


                   all        114        697      0.801      0.605       0.68       0.36
               Hardhat         42         79      0.919      0.696      0.811       0.47
                  Mask         19         21      0.937       0.81      0.842      0.519
            NO-Hardhat         37         69      0.744      0.493      0.569      0.265
               NO-Mask         44         74      0.842      0.432      0.523       0.22
        NO-Safety Vest         56        106       0.77      0.453      0.567      0.281
                Person         84        166      0.772      0.657      0.713      0.372
           Safety Cone         13         44       0.75      0.795      0.802      0.359
           Safety Vest         28         41      0.952       0.61       0.74      0.392
             machinery         26         55       0.78        0.8       0.84      0.494
               vehicle         16         42      0.547       0.31      0.396      0.226
Speed: 0.5ms preproce

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f24d2daa8d0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0

In [10]:
from ultralytics import YOLO

# List of YOLOv8 model variants
models = ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt", "yolov8l.pt"]
results = {}

# Train each model
for model_name in models:
    print(f"Training {model_name}...")
    model = YOLO(model_name)
    result = model.train(data="/content/Construction-Site-Safety-28/data.yaml", epochs=10, imgsz=640)
    results[model_name] = result


Training yolov8n.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/Construction-Site-Safety-28/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

train: Scanning /content/Construction-Site-Safety-28/train/labels.cache... 2605 images, 6 backgrounds, 0 corrupt: 100%|██████████| 2605/2605 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/004720_jpg.rf.afc486560a4004c7cfd67910af31a29c.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/Construction-Site-Safety-28/train/images/construction-813-_jpg.rf.b085952261fd98f2e76b8065de149b5f.jpg: 1 duplicate labels removed
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/Construction-Site-Safety-28/valid/labels.cache... 114 images, 10 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      10/10      13.2G     0.9486     0.7441      1.227        172        640: 100%|██████████| 163/163 [02:17<00:00,  1.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.54it/s]

                   all        114        697        0.9      0.676      0.779       0.42



10 epochs completed in 0.447 hours.
Optimizer stripped from runs/detect/train7/weights/last.pt, 87.7MB
Optimizer stripped from runs/detect/train7/weights/best.pt, 87.7MB

Validating runs/detect/train7/weights/best.pt...
Ultralytics 8.3.96 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 112 layers, 43,614,318 parameters, 0 gradients, 164.9 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.19s/it]


                   all        114        697        0.9      0.676      0.779       0.42
               Hardhat         42         79      0.944      0.759      0.858      0.471
                  Mask         19         21      0.959      0.857      0.891      0.491
            NO-Hardhat         37         69      0.971      0.565      0.694      0.334
               NO-Mask         44         74      0.824        0.5      0.636      0.309
        NO-Safety Vest         56        106      0.813      0.538      0.669      0.327
                Person         84        166      0.866      0.693      0.798      0.427
           Safety Cone         13         44          1      0.828      0.888      0.461
           Safety Vest         28         41      0.887       0.77      0.868      0.476
             machinery         26         55      0.928      0.818       0.92      0.567
               vehicle         16         42      0.811      0.429      0.566      0.338
Speed: 0.3ms preproce

In [ ]:
import matplotlib.pyplot as plt

# Initialize figure
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.flatten()

# Evaluate and plot PR curve for each model
for idx, model_name in enumerate(models):
    model = YOLO(f"/content/runs/detect/train/weights/{model_name.replace('.pt', '')}.pt")
    metrics = model.val()

    # Extract PR curve data
    pr_curve = metrics.results_dict["pr_curve"]
    recall = pr_curve["recall"]
    precision = pr_curve["precision"]

    # Plot Precision-Recall Curve
    axes[idx].plot(recall, precision, label=f"{model_name}")
    axes[idx].set_title(f"YOLOv8 {model_name.replace('.pt', '')}")
    axes[idx].set_xlabel("Recall")
    axes[idx].set_ylabel("Precision")
    axes[idx].legend()

# Show the plot
plt.tight_layout()
plt.show()
